<a href="https://colab.research.google.com/github/SamytechInc/CSFIntershipAssessment2024/blob/main/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installing the necessary libraries

!pip install torch
!pip install torchvision

In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#@title Hyperparameters

input_size = 784 # The image size is 28x28, or 784 pixels
hidden_size = 500 # the number of nodes at hidden layer
num_classes = 10 # number of output classes [0,9]
num_epochs = 20 # number of times the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # the size of a step

In [3]:
#@title Downloading the MNIST dataset

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3985237.88it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134203.94it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 847904.38it/s] 


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4392559.09it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

In [5]:
#@title Defining the model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [6]:
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [7]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [10]:
#@title Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data))

Epoch [1/20], Step [100/600], Loss: 0.1278
Epoch [1/20], Step [200/600], Loss: 0.1207
Epoch [1/20], Step [300/600], Loss: 0.0780
Epoch [1/20], Step [400/600], Loss: 0.2130
Epoch [1/20], Step [500/600], Loss: 0.1826
Epoch [1/20], Step [600/600], Loss: 0.1258
Epoch [2/20], Step [100/600], Loss: 0.1503
Epoch [2/20], Step [200/600], Loss: 0.0561
Epoch [2/20], Step [300/600], Loss: 0.0495
Epoch [2/20], Step [400/600], Loss: 0.1107
Epoch [2/20], Step [500/600], Loss: 0.0613
Epoch [2/20], Step [600/600], Loss: 0.0711
Epoch [3/20], Step [100/600], Loss: 0.0733
Epoch [3/20], Step [200/600], Loss: 0.0424
Epoch [3/20], Step [300/600], Loss: 0.0856
Epoch [3/20], Step [400/600], Loss: 0.0853
Epoch [3/20], Step [500/600], Loss: 0.0414
Epoch [3/20], Step [600/600], Loss: 0.1160
Epoch [4/20], Step [100/600], Loss: 0.0514
Epoch [4/20], Step [200/600], Loss: 0.0395
Epoch [4/20], Step [300/600], Loss: 0.0099
Epoch [4/20], Step [400/600], Loss: 0.0394
Epoch [4/20], Step [500/600], Loss: 0.0293
Epoch [4/20

In [13]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()

  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.350 %
